In [1]:
from dotenv import load_dotenv
import dspy
import mlflow

from agentic_system.agents import (
    CFEfficacyAgent,
    ToxicityScreeningAgent,
    CompoundPrioritizationAgent,
)

In [2]:
# NOTE: Start MLflow server with:
# mlflow server --backend-store-uri sqlite:///mydb.sqlite
# Tell MLflow about the server URI.
mlflow.set_tracking_uri("http://127.0.0.1:5000")
# Create a unique name for your experiment.
mlflow.set_experiment("Optimize Efficacy Tools")
mlflow.autolog()
mlflow.tracing.disable_notebook_display()

load_dotenv("../.env")
dspy.configure(
    lm=dspy.LM("gemini/gemini-2.5-pro", temperature=0.5, cache=False, rollout_id=1)
)

COMPOUND = "Luminespib"

2025/09/15 17:20:30 INFO mlflow.tracking.fluent: Autologging successfully enabled for dspy.
2025/09/15 17:20:30 INFO mlflow.tracking.fluent: Autologging successfully enabled for litellm.
2025/09/15 17:20:30 INFO mlflow.tracking.fluent: Autologging successfully enabled for openai.


In [3]:
efficacy_agent = CFEfficacyAgent(max_iters=12)
# efficacy_agent.agent.react.signature = efficacy_agent.agent.react.signature.insert(
#     0,
#     "tool_expected_info",
#     dspy.OutputField(
#         desc="TRUE or FALSE: <more info>. If FALSE, describe what information (ex databases, literature, etc) was desired from tool. Do not say FALSE if tool did not return any information (ex due to no information available), only say FALSE if tool returned information that was not useful or other information was desired from this tool call.\n\n"
#     ),
#     type_=str,
# )
efficacy_results = efficacy_agent(compound_name=COMPOUND)
print(efficacy_results)

Prediction(
    trajectory={'thought_0': "I need to determine the efficacy of Luminespib in a cardiac fibroblast assay. First, I need to identify the compound and its properties. I will start by searching for its ChEMBL ID to gather more information about its mechanism of action and bioactivities. This will help me understand if it's likely to have an effect on fibroblast activation.", 'tool_name_0': 'CHEMBL__search_chembl_id', 'tool_args_0': {'query': 'Luminespib'}, 'observation_0': '- **Compound Name:** Luminespib\n- **ChEMBL IDs:** CHEMBL3137334 (LUMINESPIB MESYLATE), CHEMBL252164 (LUMINESPIB)\n- **Summary:** Two forms of Luminespib were identified in the search.', 'thought_1': 'I have identified two ChEMBL IDs for Luminespib. I will use the base compound ID, CHEMBL252164, to investigate its mechanism of action. Understanding the mechanism is crucial for predicting its effect on cardiac fibroblast activation.', 'tool_name_1': 'CHEMBL__get_mechanisms_of_action', 'tool_args_1': {'chem

In [ ]:
# tox_agent = ToxicityScreeningAgent(max_iters=12)
# tox_results = tox_agent(compound_name=COMPOUND)
# print(tox_results)

In [ ]:
# prioritization_agent = CompoundPrioritizationAgent()
# prioritization_results = prioritization_agent(compound_name=COMPOUND)
# print(prioritization_results)